In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error as mae
import xgboost as xgb
import lightgbm as lgb

In [15]:
# df_train = pd.read_csv('../input/houseprice/╖┐╬▌╫Γ┴▐╝█╕±╘ñ▓Γ╠⌠╒╜╚ⁿ╣½┐¬╩²╛▌/train.csv')
# df_test = pd.read_csv('../input/houseprice/╖┐╬▌╫Γ┴▐╝█╕±╘ñ▓Γ╠⌠╒╜╚ⁿ╣½┐¬╩²╛▌/test.csv')

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

df_train.describe()

,ID,区域1,区域2,区域3,街道,房屋租金,服务费,供暖费用,电力基础价格,没有停车位,...,加热类型,房屋类型,邮政编码,房间数量,所处楼层,建筑楼层,最后翻新年份,建成年份,价格趋势,上传图片数
count,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,2.000000e+05,194878.000000,63507.000000,34806.000000,69183.000000,...,199881.000000,195799.000000,200000.000000,200000.000000,161736.000000,127329.000000,60166.000000,157522.000000,198679.00000,200000.000000
mean,99999.500000,5.11352,112.193435,1452.648445,8684.393950,7.050609e+02,150.694673,77.473713,89.115567,1.337482,...,2.383513,1.779018,1466.059120,2.638721,2.122106,3.573703,2013.928747,1966.374310,3.38789,9.789215
std,57735.171256,4.38307,102.461610,1570.143349,11463.243457,2.253782e+04,125.662569,161.982694,5.392918,9.362136,...,2.354078,2.481294,1383.885453,1.977911,3.731785,6.622676,10.551631,46.906557,1.96158,6.399923
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,71.430000,0.000000,...,0.000000,0.000000,0.000000,1.000000,-1.000000,0.000000,1867.000000,1007.000000,-9.17000,0.000000
25%,49999.750000,1.00000,32.000000,307.000000,1.000000,3.381000e+02,95.000000,54.000000,90.760000,1.000000,...,1.000000,0.000000,365.000000,2.000000,1.000000,2.000000,2012.000000,1950.000000,2.00000,6.000000
50%,99999.500000,5.00000,74.000000,879.000000,3136.000000,4.900000e+02,135.000000,70.000000,90.760000,1.000000,...,1.000000,1.000000,1034.000000,3.000000,2.000000,3.000000,2017.000000,1973.000000,3.39000,9.000000
75%,149999.250000,8.00000,176.000000,2064.000000,13570.250000,7.990000e+02,190.000000,90.000000,90.760000,1.000000,...,3.000000,2.000000,2169.000000,3.000000,3.000000,4.000000,2019.000000,1996.000000,4.57000,13.000000
max,199999.000000,15.00000,418.000000,8225.000000,44189.000000,9.999999e+06,25000.000000,12613.000000,90.760000,2241.000000,...,12.000000,9.000000,7218.000000,560.000000,999.000000,999.000000,2919.000000,2090.000000,12.87000,121.000000


In [16]:
df_train['上传日期_day'] = df_train['上传日期'].apply(lambda x: int(x[-2:]))
df_test['上传日期_day'] = df_test['上传日期'].apply(lambda x: int(x[-2:]))

df_train['上传日期_month'] = df_train['上传日期'].apply(lambda x: x[:2])
df_test['上传日期_month'] = df_test['上传日期'].apply(lambda x: x[:2])

df_train['上传日期_month'] = df_train['上传日期_month'].apply(lambda x: ['Fe', 'Ma', 'Oc', 'Se'].index(x))
df_test['上传日期_month'] = df_test['上传日期_month'].apply(lambda x: ['Fe', 'Ma', 'Oc', 'Se'].index(x))

df_train.drop(['上传日期'], axis=1, inplace=True)
df_test.drop(['上传日期'], axis=1, inplace=True)

df_train['可带宠物'] = df_train['可带宠物'].map({'negotiable': 0.5, 'no': 0, 'yea': 1})
df_test['可带宠物'] = df_test['可带宠物'].map({'negotiable': 0.5, 'no': 0, 'yea': 1})

for col in df_train.select_dtypes(bool).columns:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

In [17]:
index_zero = df_train.loc[df_train['房屋租金'] == 0].index
print("房租价格等于0的数量有{0}".format(len(index_zero)))
df_train.drop(index = index_zero, inplace=True)
df_train = df_train.reset_index(drop=True)

index_zero_sq = df_train.loc[df_train['居住面积'] == 0].index
print("房租价格等于0的数量有{0}".format(len(index_zero_sq)))
df_train.drop(index = index_zero_sq, inplace=True)
df_train = df_train.reset_index(drop=True)

房租价格等于0的数量有68
房租价格等于0的数量有52


In [18]:
# df_train['每平米价格'] = df_train['房屋租金'] / df_train['居住面积']
df_train['每平米价格'] = df_train['房屋租金']

df_train['房屋租金'] = np.log1p(df_train['房屋租金'])

df_train['居住面积_mean'] = df_train['居住面积'] / df_train['房间数量']
df_test['居住面积_mean'] = df_test['居住面积'] / df_test['房间数量']

df_train['居住面积'] = np.log1p(df_train['居住面积'])
df_test['居住面积'] = np.log1p(df_test['居住面积'])

df_train['服务费'] = np.log1p(df_train['服务费'])
df_test['服务费'] = np.log1p(df_test['服务费'])

df_train['区域1_每平米价格'] = df_train['区域1'].map(df_train.groupby(['区域1'])['每平米价格'].mean())
df_test['区域1_每平米价格'] = df_test['区域1'].map(df_train.groupby(['区域1'])['每平米价格'].mean())

df_train['区域2_每平米价格'] = df_train['区域2'].map(df_train.groupby(['区域2'])['每平米价格'].mean())
df_test['区域2_每平米价格'] = df_test['区域2'].map(df_train.groupby(['区域2'])['每平米价格'].mean())

df_train['邮政编码_每平米价格'] = df_train['邮政编码'].map(df_train.groupby(['邮政编码'])['每平米价格'].mean())
df_test['邮政编码_每平米价格'] = df_test['邮政编码'].map(df_train.groupby(['邮政编码'])['每平米价格'].mean())

df_train['所处楼层_div'] = df_train['所处楼层'] / (df_train['建筑楼层']+1)
df_test['所处楼层_div'] = df_test['所处楼层'] / (df_test['建筑楼层']+1)

df_train['所处楼层_max'] = (df_train['所处楼层'] == df_train['建筑楼层']).astype(int)
df_test['所处楼层_max'] = (df_test['所处楼层'] == df_test['建筑楼层']).astype(int)

df_train['建成年龄'] = 2022 - df_train['建成年份']
df_test['建成年龄'] = 2022 - df_test['建成年份']

df_train['翻新年龄'] = 2022 - df_train['最后翻新年份']
df_test['翻新年龄'] = 2022 - df_test['最后翻新年份']

In [19]:
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
data['邮政编码_价格趋势'] = data['邮政编码'].map(data.groupby(['邮政编码'])['价格趋势'].mean())
data['区域1_价格趋势'] = data['区域1'].map(data.groupby(['区域1'])['价格趋势'].mean())
data['区域2_价格趋势'] = data['区域2'].map(data.groupby(['区域2'])['价格趋势'].mean())
data['区域3_价格趋势'] = data['区域3'].map(data.groupby(['区域3'])['价格趋势'].mean())
data['街道_价格趋势'] = data['街道'].map(data.groupby(['街道'])['价格趋势'].mean())

data['邮政编码_价格趋势2'] = data['邮政编码'].map(data.groupby(['邮政编码'])['价格趋势'].max())
data['区域1_价格趋势2'] = data['区域1'].map(data.groupby(['区域1'])['价格趋势'].max())
data['区域2_价格趋势2'] = data['区域2'].map(data.groupby(['区域2'])['价格趋势'].max())
data['区域3_价格趋势2'] = data['区域3'].map(data.groupby(['区域3'])['价格趋势'].max())
data['街道_价格趋势2'] = data['街道'].map(data.groupby(['街道'])['价格趋势'].max())

data['邮政编码_价格趋势3'] = data['邮政编码'].map(data.groupby(['邮政编码'])['价格趋势'].min())
data['区域1_价格趋势3'] = data['区域1'].map(data.groupby(['区域1'])['价格趋势'].min())
data['区域2_价格趋势3'] = data['区域2'].map(data.groupby(['区域2'])['价格趋势'].min())
data['区域3_价格趋势3'] = data['区域3'].map(data.groupby(['区域3'])['价格趋势'].min())
data['街道_价格趋势3'] = data['街道'].map(data.groupby(['街道'])['价格趋势'].min())

data['邮政编码_建成年份'] = data['邮政编码'].map(data.groupby(['邮政编码'])['建成年份'].mean())
data['区域1_建成年份'] = data['区域1'].map(data.groupby(['区域1'])['建成年份'].mean())
data['区域2_建成年份'] = data['区域2'].map(data.groupby(['区域2'])['建成年份'].mean())
data['区域3_建成年份'] = data['区域3'].map(data.groupby(['区域3'])['建成年份'].mean())
data['街道_建成年份'] = data['街道'].map(data.groupby(['街道'])['建成年份'].mean())

data['邮政编码_建成年份2'] = data['邮政编码'].map(data.groupby(['邮政编码'])['建成年份'].max())
data['区域1_建成年份2'] = data['区域1'].map(data.groupby(['区域1'])['建成年份'].max())
data['区域2_建成年份2'] = data['区域2'].map(data.groupby(['区域2'])['建成年份'].max())
data['区域3_建成年份2'] = data['区域3'].map(data.groupby(['区域3'])['建成年份'].max())
data['街道_建成年份2'] = data['街道'].map(data.groupby(['街道'])['建成年份'].max())

data['邮政编码_建成年份3'] = data['邮政编码'].map(data.groupby(['邮政编码'])['建成年份'].min())
data['区域1_建成年份3'] = data['区域1'].map(data.groupby(['区域1'])['建成年份'].min())
data['区域2_建成年份3'] = data['区域2'].map(data.groupby(['区域2'])['建成年份'].min())
data['区域3_建成年份3'] = data['区域3'].map(data.groupby(['区域3'])['建成年份'].min())
data['街道_建成年份3'] = data['街道'].map(data.groupby(['街道'])['建成年份'].min())

data['邮政编码_建筑楼层'] = data['邮政编码'].map(data.groupby(['邮政编码'])['建筑楼层'].mean())
data['区域1_建筑楼层'] = data['区域1'].map(data.groupby(['区域1'])['建筑楼层'].mean())
data['区域2_建筑楼层'] = data['区域2'].map(data.groupby(['区域2'])['建筑楼层'].mean())
data['区域3_建筑楼层'] = data['区域3'].map(data.groupby(['区域3'])['建筑楼层'].mean())
data['街道_建筑楼层'] = data['街道'].map(data.groupby(['街道'])['建筑楼层'].mean())

# data['邮政编码_所处楼层'] = data['邮政编码'].map(data.groupby(['邮政编码'])['所处楼层'].mean())
# data['区域1_所处楼层'] = data['区域1'].map(data.groupby(['区域1'])['所处楼层'].mean())
# data['区域2_所处楼层'] = data['区域2'].map(data.groupby(['区域2'])['所处楼层'].mean())
# data['区域3_所处楼层'] = data['区域3'].map(data.groupby(['区域3'])['所处楼层'].mean())
# data['街道_所处楼层'] = data['街道'].map(data.groupby(['街道'])['所处楼层'].mean())
    
data['邮政编码_有阳台'] = data['邮政编码'].map(data.groupby(['邮政编码'])['有阳台'].sum())
data['邮政编码_有花园'] = data['邮政编码'].map(data.groupby(['邮政编码'])['有花园'].sum())
data['邮政编码_有电梯'] = data['邮政编码'].map(data.groupby(['邮政编码'])['有电梯'].sum())
data['邮政编码_有厨房'] = data['邮政编码'].map(data.groupby(['邮政编码'])['有厨房'].sum())
data['邮政编码_是新建筑'] = data['邮政编码'].map(data.groupby(['邮政编码'])['是新建筑'].sum())


data['区域1_有阳台'] = data['区域1'].map(data.groupby(['区域1'])['有阳台'].sum())
data['区域1_有花园'] = data['区域1'].map(data.groupby(['区域1'])['有花园'].sum())
data['区域1_有电梯'] = data['区域1'].map(data.groupby(['区域1'])['有电梯'].sum())
data['区域1_有厨房'] = data['区域1'].map(data.groupby(['区域1'])['有厨房'].sum())
data['区域1_是新建筑'] = data['区域1'].map(data.groupby(['区域1'])['是新建筑'].sum())

data['区域2_有阳台'] = data['区域2'].map(data.groupby(['区域2'])['有阳台'].sum())
data['区域2_有花园'] = data['区域2'].map(data.groupby(['区域2'])['有花园'].sum())
data['区域2_有电梯'] = data['区域2'].map(data.groupby(['区域2'])['有电梯'].sum())
data['区域2_有厨房'] = data['区域2'].map(data.groupby(['区域2'])['有厨房'].sum())
data['区域2_是新建筑'] = data['区域2'].map(data.groupby(['区域2'])['是新建筑'].sum())


data['区域3_有阳台'] = data['区域3'].map(data.groupby(['区域3'])['有阳台'].sum())
data['区域3_有花园'] = data['区域3'].map(data.groupby(['区域3'])['有花园'].sum())
data['区域3_有电梯'] = data['区域3'].map(data.groupby(['区域3'])['有电梯'].sum())
data['区域3_有厨房'] = data['区域3'].map(data.groupby(['区域3'])['有厨房'].sum())
data['区域3_是新建筑'] = data['区域3'].map(data.groupby(['区域3'])['是新建筑'].sum())


data['街道_有阳台'] = data['街道'].map(data.groupby(['街道'])['有阳台'].sum())
data['街道_有花园'] = data['街道'].map(data.groupby(['街道'])['有花园'].sum())
data['街道_有电梯'] = data['街道'].map(data.groupby(['街道'])['有电梯'].sum())
data['街道_有厨房'] = data['街道'].map(data.groupby(['街道'])['有厨房'].sum())
data['街道_是新建筑'] = data['街道'].map(data.groupby(['街道'])['是新建筑'].sum())


dict_num = dict(data['邮政编码'].value_counts())
data['邮政编码数量'] =np.log1p(data['邮政编码'].map(dict_num))
dict_num = dict(data['区域1'].value_counts())
data['区域1数量'] =np.log1p(data['区域1'].map(dict_num))
dict_num = dict(data['区域2'].value_counts())
data['区域2数量'] =np.log1p(data['区域2'].map(dict_num))
dict_num = dict(data['区域3'].value_counts())
data['区域3数量'] =np.log1p(data['区域3'].map(dict_num))
dict_num = dict(data['街道'].value_counts())
data['街道数量'] =np.log1p(data['街道'].map(dict_num))

data['邮政编码_建成年份4'] = data['建成年份'] -data['邮政编码_建成年份']
data['区域1_建成年份4'] = data['建成年份'] -data['区域1_建成年份']
data['区域2_建成年份4'] = data['建成年份'] -data['区域2_建成年份']
data['区域3_建成年份4'] = data['建成年份'] -data['区域3_建成年份']
data['街道_建成年份4'] = data['建成年份'] -data['街道_建成年份']

df_train = data[data['房屋租金'].notnull()].reset_index(drop=True)
df_test = data[data['房屋租金'].isnull()].reset_index(drop=True)

In [20]:
index_high = df_train.loc[(df_train['居住面积'] <2) | (df_train['居住面积'] >7)].index
print("房租价格超过50的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

index_high = df_train.loc[df_train['服务费'] >8].index
print("房租价格超过50的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

index_high = df_train.loc[(df_train['服务费'] >0) & (df_train['服务费'] <2)].index
print("房租价格超过50的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

index_high = df_train.loc[df_train['房屋租金'] > 9].index
print("房租价格超过50的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

index_high = df_train.loc[df_train['房屋租金'] < 3].index
print("房租价格超过50的数量有：{0}".format(len(index_high)))
df_train.drop(index = index_high, inplace=True)
df_train = df_train.reset_index(drop=True)

房租价格超过50的数量有：26
房租价格超过50的数量有：10
房租价格超过50的数量有：38
房租价格超过50的数量有：26
房租价格超过50的数量有：28


In [21]:
features = df_train.columns.unique()
features = features.drop(['房屋租金', 'ID','每平米价格'])

normal_x = df_train[features]
normal_y = df_train['房屋租金']

# stacking

In [22]:
oof_lgb = np.zeros(normal_x.shape[0])
oof_xgb = np.zeros(normal_x.shape[0])

In [23]:
test_output_df = pd.DataFrame(columns=['lgb','xgb'],index=range(df_test[features].shape[0]))
test_output_df = test_output_df.fillna(0)
test_output_df

,lgb,xgb
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
49995,0,0
49996,0,0
49997,0,0
49998,0,0


In [24]:
train_x = normal_x
train_y = normal_y

In [25]:
lgb_reg = lgb.LGBMRegressor(max_depth=8,
                           n_estimators=300,
                           subsample=0.8,
                           colsample_bytree=0.8,
                           random_state=32)

In [26]:
kfold = KFold(n_splits=2,random_state=2022,shuffle=True)
for train_index, valid_index in kfold.split(train_x,train_y):
    trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y.iloc[train_index], train_x.iloc[valid_index], train_y.iloc[valid_index]
    train_matrix = xgb.DMatrix(trn_x , label=trn_y)
    valid_matrix = xgb.DMatrix(val_x , label=val_y)
    test_x = df_test[features]
    test_matrix = xgb.DMatrix(test_x)

    params = { 
        "max_depth":8,
        'objective':'reg:linear',
        'eval_metric': 'mae',
        'n_jobs':-1,
        'learning_rate':0.1,
    }
    params['gpu_id'] = 0
    params['tree_method'] = 'gpu_hist'
    watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
    xgb_reg = xgb.train(params, train_matrix, num_boost_round=8000, evals=watchlist, verbose_eval=100, early_stopping_rounds=100)

    param = {
        "seed":1111,
        "objective": "regression_l1",
        "learning_rate" : 0.1,
        'num_leaves': 256,
        "max_bin":512,
        'bagging_fraction': 0.7,
        'feature_fraction': 0.6,
        "metric": "mae",
        "boosting": "gbdt"
    }

    lgb_train = lgb.Dataset(df_train[features], df_train['房屋租金'])
    lgb_reg = lgb.train(param, train_set=lgb_train, num_boost_round=100, evals=watchlist, verbose_eval = 100,early_stopping_rounds=30,)
    
    oof_lgb[valid_index] = lgb_reg.predict(valid_index)
    oof_xgb[valid_index] = xgb_reg.predict(valid_index)
    test_output_df['lgb'] += lgb_reg.predict(test_x)
    test_output_df['xgb'] += xgb_reg.predict(test_x)

test_output_df['lgb'] = test_output_df['lgb'] / 5
test_output_df['xgb'] = test_output_df['xgb'] / 5


[13:34:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-mae:5.19966	eval-mae:5.19950
[100]	train-mae:0.09553	eval-mae:0.11067
[200]	train-mae:0.08255	eval-mae:0.10658
[300]	train-mae:0.07357	eval-mae:0.10474
[400]	train-mae:0.06607	eval-mae:0.10356
[500]	train-mae:0.05918	eval-mae:0.10269
[600]	train-mae:0.05348	eval-mae:0.10208
[700]	train-mae:0.04881	eval-mae:0.10166
[800]	train-mae:0.04458	eval-mae:0.10135
[900]	train-mae:0.04085	eval-mae:0.10109
[1000]	train-mae:0.03725	eval-mae:0.10089
[1100]	train-mae:0.03424	eval-mae:0.10075
[1200]	train-mae:0.03136	eval-mae:0.10058
[1300]	train-mae:0.02880	eval-mae:0.10042
[1400]	train-mae:0.02646	eval-mae:0.10029
[1500]	train-mae:0.02427	eval-mae:0.10017
[1600]	train-mae:0.02239	eval-mae:0.10007
[1700]	train-mae:0.02072	eval-mae:0.10002
[1800]	train-mae:0.01900	eval-mae:0.09996
[1900]	train-mae:0.01761	eval-mae:0.0

TypeError: train() got an unexpected keyword argument 'evals'

In [ ]:
oof_df = pd.DataFrame({'lgb':oof_lgb,'xgb':oof_xgb})

In [ ]:
from sklearn.linear_model import LogisticRegression

log_r = LogisticRegression()
log_r.fit(oof_df, train_data_y)
final_pre1 = log_r.predict(test_output_df)
